<a href="https://colab.research.google.com/github/christinevu510/Drug-Classification/blob/main/Notebook%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Currently working through API drafts**

In [ ]:
#Import Packages
import requests
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import sklearn.metrics
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import pickle
import sys
import string

import os
import re
import nltk
nltk.download('stopwords')
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#URLs
anx_URL = "https://api.fda.gov/drug/label.json?search=purpose:'anxiety'&limit=10"
pain_URL = "https://api.fda.gov/drug/label.json?search=purpose:'pain'&limit=10"
dementia_URL = "https://api.fda.gov/drug/label.json?search=purpose:'dementia'&limit=100"

In [ ]:
import requests

def fetch_data_from_api(search_conditions, limit=100, total_limit=10000):
    api_endpoint = "https://api.fda.gov/drug/label.json"
    all_data = []

    try:
        while len(all_data) < total_limit:
            params = {
                'search': 'purpose:' + '+'.join(search_conditions),
                'limit': limit,
                'skip': len(all_data)  # Use skip to paginate
            }

            response = requests.get(api_endpoint, params=params)
            if response.status_code == 200:
                data = response.json()
                results = data.get("results", [])
                if not results:
                    break  # No more data available, exit the loop
                all_data.extend(results)
            else:
                print(f"Error: {response.status_code}")
                break
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    return all_data

# Usage
search_conditions = ['psychiatric', 'pain', 'cholesterol', 'hypertension', 'diabetes', 'infection', 'cancer', 'asthma', 'allergies']
retrieved_data = fetch_data_from_api(search_conditions, limit=1000, total_limit=10000)  # Adjust limit and total_limit as needed
print(len(retrieved_data))  # Print the total number of rows retrieved


10000


In [ ]:
df = pd.DataFrame(retrieved_data)

In [ ]:
df.head()

,effective_time,purpose,keep_out_of_reach_of_children,when_using,questions,pregnancy_or_breast_feeding,storage_and_handling,indications_and_usage,set_id,id,...,pharmacokinetics_table,animal_pharmacology_and_or_toxicology,clinical_studies_table,pregnancy_or_breast_feeding_table,storage_and_handling_table,other_safety_information_table,components,disposal_and_waste_handling,spl_patient_package_insert_table,controlled_substance
0,20171112,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. In case of ove...,[When using this product • take with food or m...,[Questions or comments? 1-888-309-9030],"[If pregnant or breast-feeding, ask a health p...",[Other information • each capsule contains: so...,[Uses • temporarily relieves minor aches and p...,00051a01-1c2e-4abb-b04c-548401f822c6,bdafad45-6825-4cf2-bb9a-a2ff6ad2e581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230328,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. In case of ove...,NaN,[Questions or comments? 1-800-426-9391],"[If pregnant or breast-feeding, ask a health p...",[Other information store at 25°C (77°F); excur...,[Uses temporarily relieves minor aches and pai...,0005aafa-08f8-4990-a54d-cc097195708f,32c39abb-4437-45cc-9fb8-4b20baa7c245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20141207,"[Reduces occasional pain, pressure and inflamm...",[Keep this and all medication out of reach of ...,NaN,NaN,"[If pregnant or breastfeeding, ask a health pr...",[Tamper resistant for your protection. Use onl...,[Uses: Homeopathic medicine for temporary reli...,000a466f-ae34-4a25-84ec-e97d15a9f3b7,89aa506d-64d5-4c44-8ef1-0f3a2d3ca183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220407,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. Overdose warni...,NaN,[Questions or comments? 1-800-719-9260],"[If pregnant or breast-feeding, ask a health p...",[Other information • store at 20-25 ° C (68-77...,[Uses • temporarily relieves minor aches and p...,00146b91-008d-4b16-95b0-6077f98821be,8c1bf22a-2d18-4575-8a3d-1f14521d9052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20190415,[Purposes Acetaminophen..........................,[Keep this and all drugs out of the reach of c...,[When using this product • avoid alcoholic dri...,NaN,[If pregnant or breast-feeding ask a health pr...,NaN,[Uses temporarily relieves these common cold/f...,001c9dc0-81cb-454a-a676-a5242e71d386,06e429a5-0429-4899-a0d5-37fa340ccd4e,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(10000, 100)

In [ ]:
columns_to_keep = ['purpose', 'indications_and_usage', 'active_ingredient', 'warnings']

# Create a new DataFrame with only the specified columns
new_df = df.loc[:, columns_to_keep]

In [ ]:
new_df.head()

,purpose,indications_and_usage,active_ingredient,warnings
0,[Purpose Pain reliever/fever reducer],[Uses • temporarily relieves minor aches and p...,[Active ingredient (in each capsule) Naproxen ...,[Warnings Allergy alert: Naproxen sodium may c...
1,[Purpose Pain reliever/fever reducer],[Uses temporarily relieves minor aches and pai...,[Active ingredient (in each gelcap) Acetaminop...,[Warnings Liver warning: This product contains...
2,"[Reduces occasional pain, pressure and inflamm...",[Uses: Homeopathic medicine for temporary reli...,[Drug Facts Active Ingredients: Each dose cont...,"[Warnings: If symptoms persist or worsen, cons..."
3,[Purpose Pain reliever/fever reducer],[Uses • temporarily relieves minor aches and p...,[Active ingredient (in each caplet) Acetaminop...,[Warnings Liver warning: This product contains...
4,[Purposes Acetaminophen..........................,[Uses temporarily relieves these common cold/f...,[Active Ingredients Acetaminophen 325 mg Dextr...,[Warnings Liver warning : This product contain...


In [ ]:
new_df = new_df.astype(str)

In [ ]:
# apply first round of text cleaning techniques


def clean_num_1(text):
    '''Make all text lowercase, remove text in square brackets, remove punctuation.'''
    if not isinstance(text, str):
        return ''
    text = re.sub(r'\[.*?:\]', '', text)
    text = text.lower()
    text = re.sub(r'/', '/ ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text

num1 = lambda x: clean_num_1(x)


In [ ]:
sample_text = "[Purpose Pain reliever/fever reducer]"
cleaned_text = clean_num_1(sample_text)

In [ ]:
cleaned_text

'purpose pain reliever fever reducer'

In [ ]:
cleaned_df = new_df.applymap(clean_num_1)

In [ ]:
cleaned_df.head()

,purpose,indications_and_usage,active_ingredient,warnings
0,purpose pain reliever fever reducer,uses • temporarily relieves minor aches and pa...,active ingredient in each capsule naproxen sod...,warnings allergy alert naproxen sodium may cau...
1,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pain...,active ingredient in each gelcap acetaminophen...,warnings liver warning this product contains a...
2,reduces occasional pain pressure and inflammat...,uses homeopathic medicine for temporary relief...,drug facts active ingredients each dose contai...,warnings if symptoms persist or worsen consult...
3,purpose pain reliever fever reducer,uses • temporarily relieves minor aches and pa...,active ingredient in each caplet acetaminophen...,warnings liver warning this product contains a...
4,purposes acetaminophenpain reliever fever redu...,uses temporarily relieves these common cold fl...,active ingredients acetaminophen 325 mg dextro...,warnings liver warning this product contains ...


In [ ]:
# Second round of cleaning.
def clean_num2(text):
    '''Get rid of some additional punctuation'''
    text = text.replace(u'\u2022', '')  # Replace the bullet character directly
    text = text.replace('[•]', '')  # Replace the pattern as a fallback
    return text

round2 = lambda x: clean_num2(x)


In [ ]:
clean_df = cleaned_df.applymap(clean_num2)

In [ ]:
clean_df.head()

,purpose,indications_and_usage,active_ingredient,warnings
0,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pai...,active ingredient in each capsule naproxen sod...,warnings allergy alert naproxen sodium may cau...
1,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pain...,active ingredient in each gelcap acetaminophen...,warnings liver warning this product contains a...
2,reduces occasional pain pressure and inflammat...,uses homeopathic medicine for temporary relief...,drug facts active ingredients each dose contai...,warnings if symptoms persist or worsen consult...
3,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pai...,active ingredient in each caplet acetaminophen...,warnings liver warning this product contains a...
4,purposes acetaminophenpain reliever fever redu...,uses temporarily relieves these common cold fl...,active ingredients acetaminophen 325 mg dextro...,warnings liver warning this product contains ...


In [ ]:

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")


# and now our functions
def descriptive_stats(tokens, num_tokens=5, verbose=True):
    """
    Given a list of tokens, print number of tokens, number of unique tokens,
    number of characters, lexical diversity
    (https://en.wikipedia.org/wiki/Lexical_diversity),
    and num_tokens most common tokens. Return a list with
    the number of tokens, number
    of unique tokens, lexical diversity, number of characters,
    and num_tokens most common tokens.
    """

    # Calculate the # of tokens
    num_tokens_total = len(tokens)

    # Calculate the # of unique tokens
    num_unique_tokens = len(set(tokens))

    # Calculate the total # of characters in all tokens
    num_characters = sum(len(token) for token in tokens)

    # Calculate lexical diversity
    if num_tokens_total > 0:
        lexical_diversity = num_unique_tokens / num_tokens_total
    else:
        lexical_diversity = 0.0

    # Count token frequencies
    token_counts = Counter(tokens)

    # Get the top num_tokens most common tokens
    top_tokens = token_counts.most_common(num_tokens)

    if verbose:
        print(f"There are {num_tokens_total} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print("Top {} most common tokens:".format(num_tokens))
        for token, count in top_tokens:
            print(f"{token}: {count}")

    return [num_tokens_total, num_unique_tokens, lexical_diversity, num_characters, top_tokens]



def remove_stop(tokens):
    """
    Remove stop words from a list of tokens.

    Args:
        tokens (list): List of tokens.

    Returns:
        list: List of tokens with stop words removed.
    """
    filtered_tokens = [word for word in tokens if word.lower() not in sw]
    return filtered_tokens


def remove_punctuation(text, punct_set=tw_punct) :
    return("".join([ch for ch in text if ch not in punct_set]))


def tokenize(text):
    """
    Split text on whitespace and return tokens.

    Args:
        text (str): Input text.

    Returns:
        list: List of tokens.
    """
    tokens = text.split()

    return tokens

def prepare(text, pipeline) :
    tokens = str(text)

    for transform in pipeline :
        tokens = transform(tokens)

    return(tokens)

In [ ]:
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

In [ ]:
clean_df["purpose_tokens"] = clean_df["purpose"].apply(prepare,pipeline=my_pipeline)
clean_df["purpose_num_tokens"] = clean_df["purpose_tokens"].map(len)

clean_df["indications_tokens"] = clean_df["indications_and_usage"].apply(prepare,pipeline=my_pipeline)
clean_df["indications_num_tokens"] = clean_df["indications_tokens"].map(len)

clean_df["active_tokens"] = clean_df["active_ingredient"].apply(prepare,pipeline=my_pipeline)
clean_df["active_num_tokens"] = clean_df["active_tokens"].map(len)

clean_df["warnings_tokens"] = clean_df["warnings"].apply(prepare,pipeline=my_pipeline)
clean_df["warnings_num_tokens"] = clean_df["warnings_tokens"].map(len)

In [ ]:
clean_df.head()

,purpose,indications_and_usage,active_ingredient,warnings,purpose_tokens,purpose_num_tokens,indications_tokens,indications_num_tokens,active_tokens,active_num_tokens,warnings_tokens,warnings_num_tokens
0,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pai...,active ingredient in each capsule naproxen sod...,warnings allergy alert naproxen sodium may cau...,"[purpose, pain, reliever, fever, reducer]",5,"[uses, temporarily, relieves, minor, aches, pa...",22,"[active, ingredient, capsule, naproxen, sodium...",14,"[warnings, allergy, alert, naproxen, sodium, m...",243
1,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pain...,active ingredient in each gelcap acetaminophen...,warnings liver warning this product contains a...,"[purpose, pain, reliever, fever, reducer]",5,"[uses, temporarily, relieves, minor, aches, pa...",23,"[active, ingredient, gelcap, acetaminophen, 50...",6,"[warnings, liver, warning, product, contains, ...",142
2,reduces occasional pain pressure and inflammat...,uses homeopathic medicine for temporary relief...,drug facts active ingredients each dose contai...,warnings if symptoms persist or worsen consult...,"[reduces, occasional, pain, pressure, inflamma...",6,"[uses, homeopathic, medicine, temporary, relie...",8,"[drug, facts, active, ingredients, dose, conta...",31,"[warnings, symptoms, persist, worsen, consult,...",7
3,purpose pain reliever fever reducer,uses temporarily relieves minor aches and pai...,active ingredient in each caplet acetaminophen...,warnings liver warning this product contains a...,"[purpose, pain, reliever, fever, reducer]",5,"[uses, temporarily, relieves, minor, aches, pa...",23,"[active, ingredient, caplet, acetaminophen, 50...",6,"[warnings, liver, warning, product, contains, ...",145
4,purposes acetaminophenpain reliever fever redu...,uses temporarily relieves these common cold fl...,active ingredients acetaminophen 325 mg dextro...,warnings liver warning this product contains ...,"[purposes, acetaminophenpain, reliever, fever,...",11,"[uses, temporarily, relieves, common, cold, fl...",22,"[active, ingredients, acetaminophen, 325, mg, ...",13,"[warnings, liver, warning, product, contains, ...",61


In [ ]:
all_purpose_tokens= [token for tokens_list in clean_df['purpose_tokens'] for token in tokens_list]

token_desc_purpose = descriptive_stats(all_purpose_tokens)
print(token_desc_purpose)

There are 150396 tokens in the data.
There are 5744 unique tokens in the data.
There are 1012661 characters in the data.
The lexical diversity is 0.038 in the data.
Top 5 most common tokens:
pain: 10542
reliever: 7797
purpose: 7287
fever: 5937
reducer: 5442
[150396, 5744, 0.03819250511981702, 1012661, [('pain', 10542), ('reliever', 7797), ('purpose', 7287), ('fever', 5937), ('reducer', 5442)]]


In [ ]:
all_active_tokens= [token for tokens_list in clean_df['active_tokens'] for token in tokens_list]

token_desc_active = descriptive_stats(all_active_tokens)
print(token_desc_active)

There are 222681 tokens in the data.
There are 5732 unique tokens in the data.
There are 1383204 characters in the data.
The lexical diversity is 0.026 in the data.
Top 5 most common tokens:
mg: 15050
active: 11252
ingredients: 5976
ingredient: 5568
acetaminophen: 4859
[222681, 5732, 0.025740857998661763, 1383204, [('mg', 15050), ('active', 11252), ('ingredients', 5976), ('ingredient', 5568), ('acetaminophen', 4859)]]


In [ ]:
all_indications_tokens= [token for tokens_list in clean_df['indications_tokens'] for token in tokens_list]

token_desc_indications = descriptive_stats(all_indications_tokens)
print(token_desc_indications)

There are 270909 tokens in the data.
There are 5353 unique tokens in the data.
There are 1747343 characters in the data.
The lexical diversity is 0.020 in the data.
Top 5 most common tokens:
minor: 11581
temporarily: 10029
aches: 9050
uses: 8659
pains: 6742
[270909, 5353, 0.019759402603826377, 1747343, [('minor', 11581), ('temporarily', 10029), ('aches', 9050), ('uses', 8659), ('pains', 6742)]]


In [ ]:
all_warnings_tokens= [token for tokens_list in clean_df['warnings_tokens'] for token in tokens_list]

token_desc_warnings = descriptive_stats(all_warnings_tokens)
print(token_desc_warnings)

There are 1566110 tokens in the data.
There are 5603 unique tokens in the data.
There are 9748727 characters in the data.
The lexical diversity is 0.004 in the data.
Top 5 most common tokens:
use: 48503
doctor: 32901
ask: 31374
may: 27452
product: 27446
[1566110, 5603, 0.0035776541877645886, 9748727, [('use', 48503), ('doctor', 32901), ('ask', 31374), ('may', 27452), ('product', 27446)]]
